<a href="https://colab.research.google.com/github/rpalsaxena/1000daysofcode/blob/main/KMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pytorch-lightning

In [3]:
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%ls

drive/  sample_data/


In [ ]:
transform=transforms.Compose([transforms.ToTensor(), 
                              transforms.Normalize((0.1307,), (0.3081,))])

trainset = datasets.KMNIST("./", download=True, train=True, transform=transform)
testset = datasets.KMNIST("./", download=True, train=False, transform=transform)

In [7]:
trainset, valset = random_split(trainset, [55000, 5000])

kmnist_train = DataLoader(trainset, batch_size=32)
kmnist_val = DataLoader(valset, batch_size=32)
kmnist_test = DataLoader(testset, batch_size=32)

In [8]:
class LightningMNISTClassifier(pl.LightningModule):
  def __init__(self):
    super().__init__()
    
    self.layer1 = torch.nn.Linear(28*28, 256)
    self.layer2 = torch.nn.Linear(256, 128)
    self.layer3 = torch.nn.Linear(128, 10)

  def forward(self, x):
    batch_size, channels, width, height = x.size()

    x = x.view(batch_size, -1)

    x = self.layer1(x)
    x = F.relu(x)
    x = self.layer2(x)
    x = F.relu(x)
    x = self.layer3(x)
    
    return x
    
  def cross_entropy_loss(self, logits, labels):
    return F.nll_loss(logits, labels)

  def training_step(self, train_batch, batch_idx):
    x, y = train_batch
    logits = self.forward(x)
    loss = self.cross_entropy_loss(logits, y)
    self.log('train_loss', loss)
    return loss

  def validation_step(self, val_batch, batch_idx):
      x, y = val_batch
      logits = self.forward(x)
      loss = self.cross_entropy_loss(logits, y)
      self.log('val_loss', loss)

  def configure_optimizers(self):
      optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
      return optimizer



model = LightningMNISTClassifier()
trainer = pl.Trainer()

GPU available: True, used: False
TPU available: None, using: 0 TPU cores
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:51: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)


In [ ]:
trainer.fit(model, kmnist_train, kmnist_val)


  | Name   | Type   | Params
----------------------------------
0 | layer1 | Linear | 200 K 
1 | layer2 | Linear | 32.9 K
2 | layer3 | Linear | 1.3 K 
----------------------------------
235 K     Trainable params
0         Non-trainable params
235 K     Total params
0.941     Total estimated model params size (MB)
